In [1]:
# Import and GPU Support
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import sys
import time


from NS_data import create_residual_data, create_ICBC_data
from NS_Plotting import Burgers_plot, loss_plot 

# Full PINN
from NS_PINN import PINN
from NS_PINN_train import pinn_train

# Burgers GPT-PINN
from NS_GPT_activation import P
from NS_GPT_precomp import autograd_calculations, Pt_nu_lap_vor
from NS_GPT_PINN import GPT
from NS_GPT_train import gpt_train


torch.set_default_dtype(torch.float)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Current Device: {device}")


n_train = 1000

nu = 0.025
# Domain and Data
Xi, Xf = 0.0, 2*np.pi
Yi, Yf = Xi, Xf
Ti, Tf = 0.0, 10.0
Nc, N_test     = 64, 64
Nt = 64
BC_pts, IC_pts = 100, 64

residual_data = create_residual_data(Xi, Xf, Yi, Yf, Ti, Tf, Nc, Nt, N_test)
xt_resid      = residual_data[0].to(device)
xt_test       = residual_data[1].to(device) 

ICBC_data = create_ICBC_data(Xi, Xf, Yi, Yf, Ti, Tf, Nt, BC_pts, IC_pts)
IC_xt     = ICBC_data[0].to(device)
BC_xt_bottom = ICBC_data[1].to(device)
BC_xt_top    = ICBC_data[2].to(device)
BC_xt_left   = ICBC_data[3].to(device)
BC_xt_right  = ICBC_data[4].to(device)

# Training Parameter Set
K = 100
Nx = 64
path = "../data/"
# rf_coef = np.load(path + fr"Random_NS_rf_coef_{K}_{Nx}.npy")
curl_f = np.load(path + fr'Random_NS_curl_f_{K}_{Nx}.npy').transpose(1, 0, 2)
omega0 = np.load(path + fr"Random_NS_omega0_{K}_{Nx}.npy").T



# xi_train = torch.from_numpy(rf_coef[:, :, :n_train].astype(np.float32)).to(device)
omega0 = torch.from_numpy(omega0.reshape(-1,1).astype(np.float32)).to(device)
curl_f_train = torch.from_numpy(curl_f[:, :, :n_train].astype(np.float32)).to(device)

train_final_gpt   = True
number_of_neurons = 4
loss_list         = np.ones(number_of_neurons)
print(f"Expected Final GPT-PINN Depth: {[3,number_of_neurons,1]}\n")

###############################################################################
#################################### Setup ####################################
###############################################################################

P_list = np.ones(number_of_neurons, dtype=object)

lr_adam  = 0.005
lr_lbfgs = 0.8
epochs_pinn  = 60000
# epochs_lbfgs = 1000

layers_pinn = np.array([3, 20, 20, 20, 20, 20, 20, 20, 20, 2])
tol_adam    = 1e-7

lr_gpt          = 0.005
epochs_gpt      = 2000
epochs_gpt_test = 5000
test_cases      = 200

# Save Data/Plot Options
save_data         = True
plot_pinn_loss    = False
plot_pinn_sol     = False
plot_largest_loss = False

#generate mesh to find U0-pred for the whole domain

P_resid_vorstr_values = torch.ones((xt_resid.shape[0], 2, number_of_neurons)).to(device)
P_IC_values    = torch.ones((   IC_xt.shape[0], number_of_neurons)).to(device)
P_BC_bottom    = torch.ones((   BC_xt_bottom.shape[0], 2, number_of_neurons)).to(device)
P_BC_top       = torch.ones((   BC_xt_bottom.shape[0], 2, number_of_neurons)).to(device)
P_BC_left      = torch.ones((   BC_xt_bottom.shape[0], 2, number_of_neurons)).to(device)
P_BC_right     = torch.ones((   BC_xt_bottom.shape[0], 2, number_of_neurons)).to(device)

P_t_term  = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)
P_x_term  = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)
P_xx_term = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)
P_y_term  = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)
P_yy_term = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)
P_vel     = torch.ones((xt_resid.shape[0], 2, number_of_neurons)).to(device)
P_lap_psi = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)
Pt_nu_lap_omega = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)

x_resid = xt_resid[:, 0:1].to(device)
y_resid = xt_resid[:, 1:2].to(device)
t_resid = xt_resid[:, 2:3].to(device)

IC_x = IC_xt[:, 0:1].to(device)
IC_y = IC_xt[:, 1:2].to(device)
IC_t = IC_xt[:, 2:3].to(device)
BC_x_top = BC_xt_top[:, 0:1].to(device)
BC_y_top = BC_xt_top[:, 1:2].to(device)
BC_t_top = BC_xt_top[:, 2:3].to(device)
BC_x_bottom = BC_xt_bottom[:, 0:1].to(device)
BC_y_bottom = BC_xt_bottom[:, 1:2].to(device)
BC_t_bottom = BC_xt_bottom[:, 2:3].to(device)
BC_x_left = BC_xt_left[:, 0:1].to(device)
BC_y_left = BC_xt_left[:, 1:2].to(device)
BC_t_left = BC_xt_left[:, 2:3].to(device)
BC_x_right = BC_xt_right[:, 0:1].to(device)
BC_y_right = BC_xt_right[:, 1:2].to(device)
BC_t_right = BC_xt_right[:, 2:3].to(device)

Current Device: cpu
Expected Final GPT-PINN Depth: [3, 4, 1]



In [2]:
pinn_train_times = np.ones(number_of_neurons)
gpt_train_times  = np.ones(number_of_neurons)
arg_max = 0
num_pre_neurons = 0
total_train_time_1 = time.perf_counter()
###############################################################################
################################ Training Loop ################################
###############################################################################
for i in range(number_of_neurons):
    print("******************************************************************")
    ########################### Full PINN Training ############################
    
    if i < num_pre_neurons:
        path = fr"../data/Full-PINN-Data (Burgers) (K={K})/({i+1})"
        weights = []
        bias = []
        for k in range(len(layers_pinn)-1):
            weights.append(np.loadtxt(fr"{path}/saved_w{k+1}.txt"))
            bias.append(np.loadtxt(fr"{path}/saved_b{k+1}.txt"))
        P_list[i] = P(layers_pinn, weights, bias).to(device)
        P_resid_vorstr_values[:, :, i] = P_list[i](x_resid, y_resid, t_resid).detach()
        
        layers_gpt = np.array([2, i+1, 1])
        
    else:
        
        curl_f_pinn = curl_f_train[:, :, arg_max].reshape(-1, 1)
        
        # weights_pde = 100*torch.rand(Nc, 1).to(device)
        # weights_IC  = torch.rand(Nc, 1).to(device)
        
        pinn = PINN(layers_pinn, nu, curl_f_pinn, omega0).to(device)
        pinn_train_time_1 = time.perf_counter()
        
        if (i+1 == number_of_neurons):
            print(f"Begin Final Full PINN Training: case={arg_max} (Obtaining Neuron {i+1})")
        else:
            print(f"Begin Full PINN Training: case={arg_max} (Obtaining Neuron {i+1})")
        
        pinn_losses = pinn_train(pinn, nu, xt_resid, IC_xt, BC_xt_bottom, BC_xt_top,
                                 BC_xt_left, BC_xt_right, epochs_pinn, lr_adam, tol_adam)

        pinn_train_time_2 = time.perf_counter()
        print(f"PINN Training Time: {(pinn_train_time_2-pinn_train_time_1)/3600} Hours")
        
        weights = []
        bias    = []
        
        for k in range(len(layers_pinn)-1):
            weights.append(pinn.linears[k].weight.detach().cpu())
            bias.append(pinn.linears[k].bias.detach().cpu())
        
        P_list[i] = P(layers_pinn, weights, bias).to(device)

        print(f"\nCurrent GPT-PINN Depth: [3,{i+1},1]")
        
        if (save_data):        
            path = fr"../data/Full-PINN-Data (Burgers) (K={K})/({i+1})"
            
            if not os.path.exists(path):
                os.makedirs(path)
            for k in range(len(layers_pinn)-1):
                np.savetxt(fr"{path}/saved_w{k+1}.txt", weights[k].numpy())
                np.savetxt(fr"{path}/saved_b{k+1}.txt", bias[k].numpy())
      
        if (plot_pinn_sol):
            x_test = xt_test[:,0].view(-1).cpu().detach().numpy()
            y_test = xt_test[:,1].view(-1).cpu().detach().numpy()
            t_test = xt_test[:,2].view(-1).cpu().detach().numpy()
            X_star = np.hstack((x_test[:,None], y_test[:, None], t_test[:,None]))
            u = pinn(X_star)
        
            Burgers_plot(t_test, x_test, u, title=fr"SA-PINN Solution case: {arg_max}")
    
        
    
        if (i == number_of_neurons-1) and (train_final_gpt == False):
            break

        ############################ GPT-PINN Training ############################
        layers_gpt = np.array([3, i+1, 1])
        
        P_t, P_x, P_xx, P_y, P_yy, vel, lap_psi = autograd_calculations(x_resid, y_resid, t_resid, P_list[i]) 
        Pt_nu_lap_omega[:, i][:, None] = Pt_nu_lap_vor(nu, P_t, P_xx, P_yy)
   
        P_t_term[:,i][:,None]  = P_t # torch.from_numpy(u_t)
        P_x_term[:,i][:,None]  = P_x # torch.from_numpy(u_x)
        P_xx_term[:,i][:,None] = P_xx # torch.from_numpy(u_xx)
        P_y_term[:,i][:,None]  = P_y 
        P_yy_term[:,i][:,None] = P_yy
        
        
        P_IC_values[:, i][:,None] = P_list[i](IC_x, IC_y, IC_t)[:, 0:1].detach() 
        P_BC_top[:, :, i]    = P_list[i](BC_x_top, BC_y_top, BC_t_top).detach() 
        P_BC_bottom[:, :, i] = P_list[i](BC_x_bottom, BC_y_bottom, BC_t_bottom).detach()
        P_BC_left[:, :, i]   = P_list[i](BC_x_left, BC_y_left, BC_t_left).detach()
        P_BC_right[:, :, i]  = P_list[i](BC_x_right, BC_y_right, BC_t_right).detach()
        
        P_resid_vorstr_values[:, :, i] = P_list[i](x_resid, y_resid, t_resid).detach()
        P_vel[:, :, i] = vel
        P_lap_psi[:, i][:, None] = lap_psi
        
        # Finding The Next Neuron
        largest_case = 0
        largest_loss = 0

        if (i+1 == number_of_neurons):
            print("\nBegin Final GPT-PINN Training (Largest Loss Training)")
        else:
            print(f"\nBegin GPT-PINN Training (Finding Neuron {i+2} / Largest Loss Training)")

        gpt_train_time_1 = time.perf_counter()
        for j in range(n_train):

            c_initial = torch.full((1, i+1), 1/(i+1))
            curl_f_gpt = curl_f_train[:, :, j].reshape(-1, 1)
            
            P_resid_gpt = P_resid_vorstr_values[:, :, :i+1]
            P_IC_gpt    = P_IC_values[:, :i+1]
            P_BC_bottom_gpt = P_BC_bottom[:, :, :i+1]
            P_BC_top_gpt    = P_BC_top[:, :, :i+1]
            P_BC_left_gpt  = P_BC_left[:, :, :i+1]
            P_BC_right_gpt = P_BC_right[:, :, :i+1]
            P_vel_gpt = P_vel[:, :, :i+1]
            P_x_gpt   = P_x_term[:, :i+1]
            P_y_gpt   = P_y_term[:, :i+1]
            P_lap_psi_gpt = P_lap_psi[:, :i+1]
            Pt_nu_lap_omega_gpt = Pt_nu_lap_omega[:, :i+1]

            # GPT_NN = TGPT(layers_gpt, nu, P_list[0:i+1], c_initial, u0_gpt_train, f_hat).to(device)

            GPT_NN = GPT(layers_gpt, nu, P_list[0:i+1], c_initial, omega0, curl_f_gpt, P_resid_gpt, 
                         P_IC_gpt, P_BC_bottom_gpt, P_BC_top_gpt, P_BC_left_gpt, P_BC_right_gpt, Pt_nu_lap_omega_gpt, P_vel_gpt, P_x_gpt, P_y_gpt, P_lap_psi_gpt).to(device)
            gpt_losses = gpt_train(GPT_NN, nu, xt_resid, IC_xt, BC_xt_bottom, BC_xt_top, BC_xt_left,        
                                   BC_xt_right, epochs_gpt, lr_gpt, j, largest_loss, largest_case)
            
            largest_loss = gpt_losses[0]
            largest_case = gpt_losses[1]
            arg_max = largest_case
        gpt_train_time_2 = time.perf_counter()
        print("GPT-PINN Training Completed")
        print(f"\nGPT Training Time ({i+1} Neurons): {(gpt_train_time_2-gpt_train_time_1)/3600} Hours")
        
        loss_list[i] = largest_loss
        
            
        print(f"\nLargest Loss (Using {i+1} Neurons): {largest_loss}")
        print(f"Parameter Case: {largest_case}")
total_train_time_2 = time.perf_counter()                       

###############################################################################
# Results of largest loss, parameters chosen, and times may vary based on
# the initialization of full PINN and the final loss of the full PINN
print("******************************************************************")
print("*** Full PINN and GPT-PINN Training Complete ***")
print(f"Total Training Time: {(total_train_time_2-total_train_time_1)/3600} Hours\n")
print(f"Final GPT-PINN Depth: {[2,len(P_list),1]}")

******************************************************************
Begin Full PINN Training: case=0 (Obtaining Neuron 1)
Epoch: 0 | Loss: 0.1904820054769516
Epoch: 10000 | loss: 0.0004257224209140986
Epoch: 20000 | loss: 1.5508367141592316e-06
Epoch: 30000 | loss: 5.657966539729387e-07
Epoch: 40000 | loss: 4.284432634449331e-06
Epoch: 50000 | loss: 1.2431302138793399e-06
Epoch: 60000 | loss: 9.153779501502868e-06
PINN Training Completed

PINN Training Time: 1.1804032559605548 Hours

Current GPT-PINN Depth: [3,1,1]

Begin GPT-PINN Training (Finding Neuron 2 / Largest Loss Training)
arg: 0, residual loss: 4.34956655226415e-06, total loss: 9.260070328309666e-06
arg: 1, residual loss: 5.016230352339335e-05, total loss: 5.686262738890946e-05
arg: 3, residual loss: 5.473956844070926e-05, total loss: 6.22335501248017e-05
arg: 6, residual loss: 4.877939136349596e-05, total loss: 6.34582684142515e-05
arg: 7, residual loss: 5.7861514505930245e-05, total loss: 6.678431964246556e-05
arg: 14, resid